# 🚀 Getting Started with OmicSelector2 on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kstawiski/OmicSelector2/blob/main/examples/00_getting_started_colab.ipynb)

Welcome to OmicSelector2! This notebook will help you:

✅ Install OmicSelector2 on Google Colab  
✅ Set up Google Drive integration for persistent storage  
✅ Run your first biomarker discovery analysis  
✅ Save and organize results in your Drive  

**Estimated Time**: 10-15 minutes  
**Prerequisites**: None (beginner-friendly)

---

## 📌 Step 1: Install OmicSelector2

First, let's install the package and its dependencies. This takes about 2-3 minutes.

In [ ]:
# Install OmicSelector2 from GitHub
print("📦 Installing OmicSelector2... (this takes ~2-3 minutes)")
!pip install -q git+https://github.com/kstawiski/OmicSelector2.git

# Install additional dependencies for visualization
!pip install -q matplotlib seaborn plotly scikit-learn

print("✅ Installation complete!")
print("\n🔍 Verifying installation...")

# Verify installation
try:
    import omicselector2
    print(f"✅ OmicSelector2 version: {omicselector2.__version__}")
except ImportError as e:
    print(f"❌ Installation failed: {e}")
    print("Please restart runtime and try again: Runtime > Restart runtime")

## 💾 Step 2: Mount Google Drive

Mount your Google Drive to save data, models, and results persistently.

**Important**: 
- You'll be asked to authorize access
- Your files will be saved in `My Drive/OmicSelector2/`
- Data persists even after Colab session ends

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
print("🔐 Mounting Google Drive...")
drive.mount('/content/drive', force_remount=False)
print("✅ Drive mounted successfully!\n")

# Create organized folder structure
BASE_DIR = '/content/drive/MyDrive/OmicSelector2'
DIRS = {
    'data': f'{BASE_DIR}/data',
    'results': f'{BASE_DIR}/results',
    'models': f'{BASE_DIR}/models',
    'plots': f'{BASE_DIR}/plots',
    'checkpoints': f'{BASE_DIR}/checkpoints'
}

# Create directories if they don't exist
for name, path in DIRS.items():
    os.makedirs(path, exist_ok=True)
    print(f"📁 {name.capitalize()}: {path}")

print("\n✅ Folder structure created in your Google Drive!")
print(f"\n📂 Your project folder: {BASE_DIR}")
print("\nYou can access these folders from Google Drive on any device.")

## 📊 Step 3: Create Sample Dataset

Let's create a synthetic gene expression dataset and save it to Drive.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

# Set random seed for reproducibility
np.random.seed(42)

print("🧬 Generating synthetic gene expression data...\n")

# Generate data: 200 samples, 500 genes, 20 truly informative
X, y = make_classification(
    n_samples=200,
    n_features=500,
    n_informative=20,
    n_redundant=30,
    n_classes=2,
    flip_y=0.02,
    random_state=42
)

# Convert to DataFrame with gene names
gene_names = [f"GENE_{i:04d}" for i in range(X.shape[1])]
sample_ids = [f"SAMPLE_{i:03d}" for i in range(X.shape[0])]

X_df = pd.DataFrame(X, columns=gene_names, index=sample_ids)
y_df = pd.DataFrame({'sample_id': sample_ids, 'response': y, 'response_label': ['Responder' if yi == 1 else 'Non-responder' for yi in y]})

print(f"✅ Dataset created:")
print(f"   • Samples: {X_df.shape[0]}")
print(f"   • Genes: {X_df.shape[1]}")
print(f"   • Response distribution: {y_df['response_label'].value_counts().to_dict()}")

# Save to Google Drive
print(f"\n💾 Saving to Google Drive...")
X_df.to_csv(f"{DIRS['data']}/gene_expression.csv")
y_df.to_csv(f"{DIRS['data']}/clinical_data.csv", index=False)

print(f"✅ Data saved to:")
print(f"   • {DIRS['data']}/gene_expression.csv")
print(f"   • {DIRS['data']}/clinical_data.csv")
print(f"\n💡 These files are now in your Google Drive and will persist after this session ends!")

# Display first few rows
print("\n📋 Preview of gene expression data:")
display(X_df.iloc[:5, :5])

print("\n📋 Preview of clinical data:")
display(y_df.head())

## 🔬 Step 4: Your First Feature Selection

Let's identify the most important genes using Random Forest.

In [ ]:
from omicselector2.features.classical.random_forest import RandomForestSelector
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

print("🔍 Running feature selection...\n")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples\n")

# Feature selection
selector = RandomForestSelector(
    n_estimators=100,
    n_features_to_select=30,
    task='classification',
    random_state=42
)

print("⏳ Training Random Forest selector...")
selector.fit(X_train, y_train)
print("✅ Feature selection complete!\n")

# Get results
selected_genes = selector.selected_features_
print(f"🎯 Selected {len(selected_genes)} important genes\n")
print("Top 10 genes:")
for i, gene in enumerate(selected_genes[:10], 1):
    print(f"  {i:2d}. {gene}")

# Get feature importance
result = selector.get_result()
importance_df = result.to_dataframe()

# Visualize
plt.figure(figsize=(10, 6))
top_20 = importance_df.head(20)
plt.barh(range(len(top_20)), top_20['importance'], color='steelblue')
plt.yticks(range(len(top_20)), top_20['feature'])
plt.xlabel('Feature Importance', fontsize=12)
plt.ylabel('Gene', fontsize=12)
plt.title('Top 20 Most Important Genes', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()

# Save plot to Drive
plot_path = f"{DIRS['plots']}/feature_importance.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"\n💾 Plot saved to: {plot_path}")
plt.show()

# Save selected genes to Drive
selected_genes_df = pd.DataFrame({
    'rank': range(1, len(selected_genes) + 1),
    'gene': selected_genes,
    'importance': importance_df['importance'].values
})
results_path = f"{DIRS['results']}/selected_genes.csv"
selected_genes_df.to_csv(results_path, index=False)
print(f"💾 Selected genes saved to: {results_path}")

## 🤖 Step 5: Train and Evaluate a Model

Train a Random Forest classifier using the selected genes.

In [ ]:
from omicselector2.models.classical import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import pickle

print("🤖 Training model with selected genes...\n")

# Transform data using selected genes
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Train classifier
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train_selected, y_train)
print("✅ Model trained!\n")

# Predictions
y_pred = model.predict(X_test_selected)
y_pred_proba = model.predict_proba(X_test_selected)[:, 1]

# Evaluation
auc = roc_auc_score(y_test, y_pred_proba)
print("📊 Performance Metrics:")
print("=" * 50)
print(classification_report(y_test, y_pred, target_names=['Non-responder', 'Responder']))
print(f"\n🎯 AUC-ROC Score: {auc:.3f}")

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'Model (AUC = {auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save ROC curve to Drive
roc_path = f"{DIRS['plots']}/roc_curve.png"
plt.savefig(roc_path, dpi=300, bbox_inches='tight')
print(f"\n💾 ROC curve saved to: {roc_path}")
plt.show()

# Save model to Drive
model_path = f"{DIRS['models']}/random_forest_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump({'model': model, 'selector': selector}, f)
print(f"💾 Model saved to: {model_path}")
print("\n✅ You can load this model later and use it for predictions!")

## 📁 Step 6: Organize Your Results

Let's create a summary report and save it to Drive.

In [ ]:
from datetime import datetime

# Create summary report
report = f"""
═══════════════════════════════════════════════════════════════
        OmicSelector2 - Biomarker Discovery Report
═══════════════════════════════════════════════════════════════

Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET INFORMATION
───────────────────────────────────────────────────────────────
Total Samples:        {X_df.shape[0]}
Total Genes:          {X_df.shape[1]}
Training Samples:     {X_train.shape[0]}
Test Samples:         {X_test.shape[0]}

FEATURE SELECTION RESULTS
───────────────────────────────────────────────────────────────
Method:               Random Forest Variable Importance
Selected Features:    {len(selected_genes)}
Top 10 Genes:         {', '.join(selected_genes[:10])}

MODEL PERFORMANCE
───────────────────────────────────────────────────────────────
Model:                Random Forest Classifier
AUC-ROC Score:        {auc:.4f}

FILES SAVED TO GOOGLE DRIVE
───────────────────────────────────────────────────────────────
Data:                 {DIRS['data']}/gene_expression.csv
                      {DIRS['data']}/clinical_data.csv
Results:              {DIRS['results']}/selected_genes.csv
Model:                {DIRS['models']}/random_forest_model.pkl
Plots:                {DIRS['plots']}/feature_importance.png
                      {DIRS['plots']}/roc_curve.png

NEXT STEPS
───────────────────────────────────────────────────────────────
✓ Your data, models, and results are saved in Google Drive
✓ You can access them from any device
✓ Share the Drive folder with collaborators
✓ Try the other tutorial notebooks for advanced features

═══════════════════════════════════════════════════════════════
"""

print(report)

# Save report to Drive
report_path = f"{DIRS['results']}/analysis_report.txt"
with open(report_path, 'w') as f:
    f.write(report)

print(f"\n💾 Report saved to: {report_path}")

## 🎓 What You've Learned

Congratulations! You've completed your first biomarker discovery analysis with OmicSelector2! 🎉

### In this notebook, you:

✅ Installed OmicSelector2 on Google Colab  
✅ Set up Google Drive for persistent storage  
✅ Created and saved a gene expression dataset  
✅ Performed feature selection to identify important genes  
✅ Trained a machine learning model  
✅ Evaluated model performance with ROC curves  
✅ Saved all results to your Google Drive  

### Your Google Drive Structure

```
My Drive/
└── OmicSelector2/
    ├── data/               # Your datasets
    ├── results/            # Analysis results
    ├── models/             # Trained models
    ├── plots/              # Visualizations
    └── checkpoints/        # For long-running analyses
```

### 📚 Next Steps: Explore More Tutorials

Continue your learning journey with these notebooks:

1. **[01_basic_feature_selection.ipynb](01_basic_feature_selection.ipynb)** - Compare multiple feature selection methods
2. **[02_hyperparameter_tuning.ipynb](02_hyperparameter_tuning.ipynb)** - Optimize model performance
3. **[03_signature_benchmarking.ipynb](03_signature_benchmarking.ipynb)** - Compare biomarker signatures
4. **[04_data_preprocessing.ipynb](04_data_preprocessing.ipynb)** - Work with your own data
5. **[05_stability_selection.ipynb](05_stability_selection.ipynb)** - Robust feature selection
6. **[06_model_interpretation.ipynb](06_model_interpretation.ipynb)** - Understand model predictions
7. **[07_complete_workflow.ipynb](07_complete_workflow.ipynb)** - End-to-end analysis

### 💡 Tips for Using Google Drive with Colab

1. **Persistence**: Your Drive files persist across sessions
2. **Sharing**: Share your Drive folder with collaborators
3. **Backup**: Drive automatically backs up your work
4. **Access**: Open files from any device
5. **Organization**: Keep your project organized in folders

### 🆘 Need Help?

- 📖 [Documentation](https://github.com/kstawiski/OmicSelector2)
- 💬 [GitHub Discussions](https://github.com/kstawiski/OmicSelector2/discussions)
- 🐛 [Report Issues](https://github.com/kstawiski/OmicSelector2/issues)

---

**Happy biomarker hunting! 🧬🔬**